In [10]:
# workspace = 'dev'
# apiVersion = "5.2"
# moduleName = "Ed-Fi5"
# schoolYear = None
# districtId = None

StatementMeta(spark3p3sm, 209, 21, Finished, Available)

In [11]:
%run /edfi_py

StatementMeta(, 209, -1, Finished, Available)

2023-07-24 14:11:55,485 - OEA - INFO - Now using workspace: dev
2023-07-24 14:11:55,486 - OEA - INFO - OEA initialized.
2023-07-24 14:11:56,280 - OEA - INFO - Now using workspace: dev
2023-07-24 14:11:56,762 - EdFiClient - INFO - minChangeVersion=None and maxChangeVersion=None


In [12]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"

# parameterized = True

StatementMeta(spark3p3sm, 209, 28, Finished, Available)

In [13]:
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi/parameterizedRunLogs/currentRun'
    tpdmEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm/parameterizedRunLogs/currentRun'

    _, edfiEntities = edfi.listSpecifiedEntities(edfiEntitiesPath)
    _, tpdmEntities = edfi.listSpecifiedEntities(tpdmEntitiesPath)
else:
    edfiEntities = "All"
    tpdmEntities = "All"   

StatementMeta(spark3p3sm, 209, 29, Finished, Available)

In [14]:
def refine_edfi_dataset(tables_source,
                        items = 'All'):
    full_source_path = f"stage2/Ingested/{tables_source}"
    if items == 'All':
        items = oea.get_folders(full_source_path)          
    for item in items: 
        table_path = f"stage2/Ingested/{tables_source}/{item}"
        if item == 'metadata.csv' or item == 'parameterizedRunLogs':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        else:
            try:
                oea.refine(f'{tables_source}/{item}', metadata[item], 'id')
            except AnalysisException as e:
                # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
                logger.info(e)
                pass
            
            logger.info('Refined table: ' + item + ' from: ' + table_path)

StatementMeta(spark3p3sm, 209, 30, Finished, Available)

In [15]:
metadata = oea.get_metadata_from_url(metadataUrl)

refine_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi',
                    edfiEntities)
refine_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm',
                    tpdmEntities)

StatementMeta(spark3p3sm, 209, 31, Finished, Available)

2023-07-24 14:12:58,538 - EdFiClient - INFO - No updated rows in stage2/Ingested/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors to process.
2023-07-24 14:12:59,489 - EdFiClient - INFO - Refined table: absenceEventCategoryDescriptors from: stage2/Ingested/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors
2023-07-24 14:13:09,607 - EdFiClient - INFO - No updated rows in stage2/Ingested/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors to process.
2023-07-24 14:13:10,255 - EdFiClient - INFO - Refined table: academicHonorCategoryDescriptors from: stage2/Ingested/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors
